In [ ]:
import json
import pandas as pd
import time


# Load the CSV file into a Pandas DataFrame
df = pd.read_csv("/home/ethereum/SNCS/SNCS_Data/ProcessedData/order100K.csv")
# df = df.head(10000)
df.head()

In [ ]:
df.shape

# Naive Analysis

In [ ]:
# # Record the start time for performance measurement
# start_time = time.perf_counter()

# # Set the threshold for free delivery
# threshold = 200000

# # Initialize a list to store free delivery status for each order
# free_delivery = []

# # Loop through each row in the DataFrame to check if free delivery is applicable
# for i in range(df.shape[0]):
#     if df.iloc[i, 2] >= threshold:
#         free_delivery.append("yes")
#     else:
#         free_delivery.append("no")

# # Add a new column for free delivery status to the DataFrame
# df["free_delivery"] = free_delivery

# # Save the DataFrame to a CSV file
# df.to_csv("naiveReward10K.csv", index=False)

# # Record the end time and calculate the processing time
# end_time = time.perf_counter()
# processing_time = end_time - start_time

# # Load existing log data from a JSON file
# with open("log.json") as file:
#     data = json.load(file)

# # Add the processing time for the naive approach to the log data
# data["10K - naive"] = processing_time

# # Save the updated log data to the JSON file
# with open("log.json", 'w') as file:
#     json.dump(data, file)


# HiShip Analysis

In [ ]:
# Initialize a new column with default value "no"
start_time = time.perf_counter()
column = []
for i in range(df.shape[0]):
    column.append("no")


df["o_freedelivery"] = column  # Add the new column to the DataFrame


# Function to check if free delivery is applicable based on total spent
def check_freedelivery(total_spent, o_totalprice):
    threshold = 200000
    if (total_spent + float(o_totalprice) >= threshold):
        return "yes"
    else:
        return "no"


def calc_total(o_custkey, o_orderdate, chain):
    total_spent = 0
    for i in range(chain.shape[0]-1, -1, -1):
        tx_date = chain.iloc[i, 3]
        spent_time = int(o_orderdate[:4]) - int(tx_date[:4]) + int(o_orderdate[5:7])/12 - int(tx_date[5:7])/12 + int(o_orderdate[8:])/365 - int(tx_date[8:])/365
        tx_price = round(chain.iloc[i, 2])
        tx_custkey = chain.iloc[i, 1]
        tx_freedelivery = chain.iloc[i, 4]
        
        # Check if the transaction is within 1 year and matches the customer and does not have free delivery
        if spent_time <= 1:
            if o_custkey == tx_custkey:
                if tx_freedelivery == "no":
                    total_spent += tx_price
                else:
                    return total_spent
        else:
            return total_spent
    return total_spent

# Loop through each row in the DataFrame to calculate total spent and check free delivery eligibility
for tx_no in range(df.shape[0]):
    total_spent = calc_total(df.iloc[tx_no, 1], df.iloc[tx_no, 3], df.iloc[:tx_no])
    print(tx_no+1, ":", total_spent, "=>", check_freedelivery(total_spent, df.iloc[tx_no, 2]))
    df.iloc[tx_no, 4] = check_freedelivery(total_spent, df.iloc[tx_no, 2])

# Save the DataFrame to a CSV file
df.to_csv("chainReward100K.csv", index=False)


end_time = time.perf_counter()
processing_time = end_time - start_time
with open("log.json") as file:
    data = json.load(file)

data["100K - HIShip"] = processing_time

# Save the updated log data to the JSON file
with open("log.json", 'w') as file:
    json.dump(data, file)



In [ ]:
count = 0
for i in df["O_FREEDELIVERY"]:
    if i == "yes":
        count += 1
count